# TODO: Title

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [2]:
from sagemaker.tuner import HyperparameterTuner
from sagemaker.parameter import ContinuousParameter, IntegerParameter

In [3]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install -U sagemaker


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [5]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
import boto3

## Dataset
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

In [6]:
# #TODO: Fetch and upload the data to AWS S3

# # Command to download and unzip data
# !wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
# !unzip dogImages.zip

In [7]:
ls

CODEOWNERS   dogImages/     model.pth                     train_model.py
LICENSE.txt  dogImages.zip  train_and_deploy-Copy1.ipynb
README.md    hpo.py         train_and_deploy.ipynb


## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [8]:
#TODO: Declare your HP ranges, metrics etc.
hyperparameter_ranges = {
    "lr": ContinuousParameter(0.1, 1),
}
objective_metric_name = "validation:loss"



In [9]:
#TODO: Create estimators for your HPs
from sagemaker.estimator import Estimator
from sagemaker import image_uris
from sagemaker.pytorch import PyTorch



sess = sagemaker.Session()
region = sess.boto_region_name
role = sagemaker.get_execution_role()

bucket = sess.default_bucket()


estimator = PyTorch(
    entry_point="hpo.py",
    role=role,
    py_version="py310",
    framework_version="2.0.0",
    instance_count=2,
    instance_type="ml.c5.2xlarge"
)


tuner = HyperparameterTuner(
    estimator = estimator,
    objective_metric_name = objective_metric_name,
    metric_definitions = [{'Name': 'validation:loss',
                       'Regex': 'loss: (\S+)'}],
    hyperparameter_ranges  = hyperparameter_ranges,
    objective_type="Minimize",
    max_jobs=5,
    max_parallel_jobs=1,
)      

In [10]:
# TODO: Fit your HP Tuner
train_inputs = {
    "train": "s3://udacity-demo123/dogImages/train",
    "validate": "s3://udacity-demo123/dogImages/valid"
}

# tuner.fit(inputs=train_inputs, estimator_kwargs = {"num-samples": 50})    # TODO: Remember to include your data channels

In [12]:
# # TODO: Get the best estimators and the best HPs

# best_estimator = tuner.best_estimator()

# #Get the hyperparameters of the best trained model
# best_estimator.hyperparameters()

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [15]:
# TODO: Set up debugging and profiling rules and hooks

from sagemaker.debugger import CollectionConfig
from sagemaker.debugger import DebuggerHookConfig
from sagemaker.debugger import Rule, ProfilerRule, rule_configs
from sagemaker.debugger import ProfilerConfig, FrameworkProfile

debugger_profiler_rules = [
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
    
]


hook_config = DebuggerHookConfig(
    hook_parameters={"train.save_interval": "100", "eval.save_interval": "10"}
)


profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=10)
)




See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [17]:
# TODO: Create and fit an estimator

estimator = PyTorch(
    entry_point="train_model.py",
    role=role,
    py_version="py310",
    framework_version="2.0.0",
    instance_count=2,
    instance_type="ml.c5.2xlarge",
    debugger_hook_config=hook_config,
    profiler_config=profiler_config,
    rules = debugger_profiler_rules
)

estimator.fit(train_inputs)


See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2023-09-01-02-19-30-380


Using provided s3_resource
2023-09-01 02:19:30 Starting - Starting the training job...
2023-09-01 02:19:46 Starting - Preparing the instances for trainingVanishingGradient: InProgress
Overfit: InProgress
Overtraining: InProgress
PoorWeightInitialization: InProgress
LossNotDecreasing: InProgress
ProfilerReport: InProgress
LowGPUUtilization: InProgress
......
2023-09-01 02:20:50 Downloading - Downloading input data......
2023-09-01 02:22:01 Training - Downloading the training image...
2023-09-01 02:22:23 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-09-01 02:22:31,538 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-09-01 02:22:31,538 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-09-01 02:22:31,539 sagemaker-training-toolkit INFO     No Neurons detected (normal i

In [ ]:
from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys

trial = create_trial(estimator.latest_job_debugger_artifacts_path())

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [28]:
rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"

In [30]:
! aws s3 ls {rule_output_path} --recursive


2023-09-01 02:25:29     330076 pytorch-training-2023-09-01-02-19-30-380/rule-output/ProfilerReport/profiler-output/profiler-report.html
2023-09-01 02:25:29     171464 pytorch-training-2023-09-01-02-19-30-380/rule-output/ProfilerReport/profiler-output/profiler-report.ipynb
2023-09-01 02:25:25        191 pytorch-training-2023-09-01-02-19-30-380/rule-output/ProfilerReport/profiler-output/profiler-reports/BatchSize.json
2023-09-01 02:25:25        199 pytorch-training-2023-09-01-02-19-30-380/rule-output/ProfilerReport/profiler-output/profiler-reports/CPUBottleneck.json
2023-09-01 02:25:25        126 pytorch-training-2023-09-01-02-19-30-380/rule-output/ProfilerReport/profiler-output/profiler-reports/Dataloader.json
2023-09-01 02:25:25        127 pytorch-training-2023-09-01-02-19-30-380/rule-output/ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.json
2023-09-01 02:25:25        198 pytorch-training-2023-09-01-02-19-30-380/rule-output/ProfilerReport/profiler-output/profiler-re

In [31]:
! aws s3 cp {rule_output_path} ./ --recursive


download: s3://sagemaker-ap-southeast-2-847101187428/pytorch-training-2023-09-01-02-19-30-380/rule-output/ProfilerReport/profiler-output/profiler-reports/BatchSize.json to ProfilerReport/profiler-output/profiler-reports/BatchSize.json
download: s3://sagemaker-ap-southeast-2-847101187428/pytorch-training-2023-09-01-02-19-30-380/rule-output/ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.json to ProfilerReport/profiler-output/profiler-reports/GPUMemoryIncrease.json
download: s3://sagemaker-ap-southeast-2-847101187428/pytorch-training-2023-09-01-02-19-30-380/rule-output/ProfilerReport/profiler-output/profiler-report.html to ProfilerReport/profiler-output/profiler-report.html
download: s3://sagemaker-ap-southeast-2-847101187428/pytorch-training-2023-09-01-02-19-30-380/rule-output/ProfilerReport/profiler-output/profiler-reports/Dataloader.json to ProfilerReport/profiler-output/profiler-reports/Dataloader.json
download: s3://sagemaker-ap-southeast-2-847101187428/pytorch-tra

## Model Deploying

In [19]:
# TODO: Deploy your model to an endpoint

# training_job_name = estimator.latest_training_job.name
# attached_estimator = Estimator.attach(training_job_name)

predictor = estimator.deploy(initial_instance_count=1, instance_type="ml.c5.2xlarge")


INFO:sagemaker:Repacking model artifact (s3://sagemaker-ap-southeast-2-847101187428/pytorch-training-2023-09-01-02-19-30-380/output/model.tar.gz), script artifact (s3://sagemaker-ap-southeast-2-847101187428/pytorch-training-2023-09-01-02-19-30-380/source/sourcedir.tar.gz), and dependencies ([]) into single tar.gz file located at s3://sagemaker-ap-southeast-2-847101187428/pytorch-training-2023-09-01-03-10-58-084/model.tar.gz. This may take some time depending on model size...
INFO:sagemaker:Creating model with name: pytorch-training-2023-09-01-03-10-58-084
INFO:sagemaker:Creating endpoint-config with name pytorch-training-2023-09-01-03-10-58-084
INFO:sagemaker:Creating endpoint with name pytorch-training-2023-09-01-03-10-58-084


----!

In [22]:
# TODO: Run an prediction on the endpoint

import torch
from torchvision import transforms
from PIL import Image, ImageFile
# Load the saved transformation parameters
saved_transform_params = {
    'mean': [0.485, 0.456, 0.406],
    'std': [0.229, 0.224, 0.225]
}

# Define the transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(saved_transform_params['mean'], saved_transform_params['std'])
])

# Load and preprocess the image
image = Image.open('dogImages/test/006.American_eskimo_dog/American_eskimo_dog_00415.jpg')
preprocessed_image = transform(image)



In [24]:
preprocessed_image

tensor([[[-1.5528, -1.5014, -1.5699,  ..., -1.7069, -1.6898, -1.6555],
         [-1.5870, -1.5528, -1.5699,  ..., -1.6213, -1.6555, -1.6213],
         [-1.6213, -1.5528, -1.5357,  ..., -1.6213, -1.6384, -1.6213],
         ...,
         [-2.0152, -1.8268, -1.4329,  ..., -0.4911, -0.5596, -0.7308],
         [-1.9467, -1.6898, -1.6727,  ..., -0.8507, -0.9020, -1.1247],
         [-1.8097, -1.4672, -1.7754,  ..., -1.1075, -1.4843, -1.7412]],

        [[-1.6331, -1.6155, -1.5630,  ..., -1.4930, -1.4755, -1.4230],
         [-1.6506, -1.6155, -1.5630,  ..., -1.4755, -1.4755, -1.4230],
         [-1.5980, -1.5805, -1.5455,  ..., -1.5630, -1.5105, -1.4405],
         ...,
         [-1.9832, -1.7906, -1.3354,  ..., -0.1975, -0.0399, -0.3025],
         [-1.9307, -1.6856, -1.5105,  ..., -0.4251, -0.3550, -0.6176],
         [-1.6506, -1.3880, -1.4405,  ..., -0.5301, -0.8803, -1.1779]],

        [[-0.4624, -0.3753, -0.4101,  ..., -0.7936, -0.7761, -0.8110],
         [-0.5321, -0.4450, -0.4275,  ..., -0

In [ ]:
response = predictor.predict(preprocessed_image)

print(f"response: {response}")

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()